In [2]:
import pandas as pd

df_train = pd.read_csv('/home/ankur/Desktop/ML_DL_Projects/data/cmi-detect-behavior-with-sensor-data/train.csv')
df_train_demographics = pd.read_csv('/home/ankur/Desktop/ML_DL_Projects/data/cmi-detect-behavior-with-sensor-data/train_demographics.csv')
df_test = pd.read_csv('/home/ankur/Desktop/ML_DL_Projects/data/cmi-detect-behavior-with-sensor-data/test.csv')
df_test_demographics = pd.read_csv('/home/ankur/Desktop/ML_DL_Projects/data/cmi-detect-behavior-with-sensor-data/test_demographics.csv')
df_merged_train  = pd.merge(df_train, df_train_demographics, on='subject', how='left')

df_test = pd.read_csv('/home/ankur/Desktop/ML_DL_Projects/data/cmi-detect-behavior-with-sensor-data/test.csv')
df_test_demographics = pd.read_csv('/home/ankur/Desktop/ML_DL_Projects/data/cmi-detect-behavior-with-sensor-data/test_demographics.csv')
merged_test = pd.merge(df_test, df_test_demographics, on='subject', how='left')

In [3]:
test = merged_test.drop(columns=['row_id','subject']) 
test = test.sort_values(['sequence_id', 'sequence_counter'], ascending=[True, False])
# Fill null values based on your rules
test['age'] = test['age'].fillna(test['age'].mean())
test['sex'] = test['sex'].fillna(test['sex'].mode()[0])
test['adult_child'] = test['adult_child'].fillna(test['adult_child'].mode()[0])

test['handedness'] = test['handedness'].fillna(test['handedness'].mode()[0])
test['height_cm'] = test['height_cm'].fillna(test['height_cm'].mean())
test['shoulder_to_wrist_cm'] = test['shoulder_to_wrist_cm'].fillna(test['shoulder_to_wrist_cm'].mean())
test['elbow_to_wrist_cm'] = test['elbow_to_wrist_cm'].fillna(test['elbow_to_wrist_cm'].mean())

df = test

# Find target columns
prefixes = ("acc", "rot", "thm", "tof")
target_cols = [col for col in df.columns if col.startswith(prefixes)]

# Fill NaNs
for col in target_cols:
    for idx in df.index[df[col].isna()]:
        prev_val = df[col].iloc[idx-1] if idx > 0 else np.nan
        next_val = df[col].iloc[idx+1] if idx < len(df)-1 else np.nan

        if not np.isnan(prev_val) and not np.isnan(next_val):
            df.loc[idx, col] = (prev_val + next_val) / 2
        elif not np.isnan(prev_val):
            df.loc[idx, col] = prev_val
        elif not np.isnan(next_val):
            df.loc[idx, col] = next_val

test = df


In [4]:
feature_list_with_seq_id_seq_counter  =test.columns.to_list()
feature_list_with_seq_id_seq_counter

['sequence_id',
 'sequence_counter',
 'acc_x',
 'acc_y',
 'acc_z',
 'rot_w',
 'rot_x',
 'rot_y',
 'rot_z',
 'thm_1',
 'thm_2',
 'thm_3',
 'thm_4',
 'thm_5',
 'tof_1_v0',
 'tof_1_v1',
 'tof_1_v2',
 'tof_1_v3',
 'tof_1_v4',
 'tof_1_v5',
 'tof_1_v6',
 'tof_1_v7',
 'tof_1_v8',
 'tof_1_v9',
 'tof_1_v10',
 'tof_1_v11',
 'tof_1_v12',
 'tof_1_v13',
 'tof_1_v14',
 'tof_1_v15',
 'tof_1_v16',
 'tof_1_v17',
 'tof_1_v18',
 'tof_1_v19',
 'tof_1_v20',
 'tof_1_v21',
 'tof_1_v22',
 'tof_1_v23',
 'tof_1_v24',
 'tof_1_v25',
 'tof_1_v26',
 'tof_1_v27',
 'tof_1_v28',
 'tof_1_v29',
 'tof_1_v30',
 'tof_1_v31',
 'tof_1_v32',
 'tof_1_v33',
 'tof_1_v34',
 'tof_1_v35',
 'tof_1_v36',
 'tof_1_v37',
 'tof_1_v38',
 'tof_1_v39',
 'tof_1_v40',
 'tof_1_v41',
 'tof_1_v42',
 'tof_1_v43',
 'tof_1_v44',
 'tof_1_v45',
 'tof_1_v46',
 'tof_1_v47',
 'tof_1_v48',
 'tof_1_v49',
 'tof_1_v50',
 'tof_1_v51',
 'tof_1_v52',
 'tof_1_v53',
 'tof_1_v54',
 'tof_1_v55',
 'tof_1_v56',
 'tof_1_v57',
 'tof_1_v58',
 'tof_1_v59',
 'tof_1_v60',

In [5]:
test.shape

(107, 341)

In [6]:
import numpy as np
import pandas as pd

def prepare_sequences(df, max_steps=700):
    # Drop unnecessary columns early
    df = df.sort_values("sequence_counter", ascending=False)
    df = df.drop(columns=["sequence_counter"])  
    
    # Get number of features (exclude seq_id)
    feature_cols = [c for c in df.columns if c != "sequence_id"]
    n_features = len(feature_cols)
    
    # Find number of unique sequences
    sequence_ids = df["sequence_id"].unique()
    n_sequences = len(sequence_ids)

    # Preallocate final array (float32 to save memory)
    X = np.zeros((n_sequences, max_steps, n_features), dtype=np.float32)
    
    for i, seq_id in enumerate(sequence_ids):
        seq = df[df["sequence_id"] == seq_id][feature_cols].values.astype(np.float32)
        
        # Truncate if longer than max_steps, keep first 700
        if len(seq) > max_steps:
            seq = seq[:max_steps]
        
        # Pad if shorter
        X[i, :len(seq), :] = seq
    
    return X

test = prepare_sequences(test)

In [7]:
test.shape

(2, 700, 339)

In [8]:
df_merged_train["sequence_type"].value_counts()

sequence_type
Target        344058
Non-Target    230887
Name: count, dtype: int64

In [9]:
print(df_merged_train["sequence_type"].unique())

['Target' 'Non-Target']


In [10]:
df_merged_train["sequence_type_encoded"] = (
    df_merged_train["sequence_type"]
    .astype(str)
    .str.strip()
    .str.title()
    .map({"Target": 1, "Non-Target": 0})
)

print(df_merged_train[["sequence_type", "sequence_type_encoded"]].head())

  sequence_type  sequence_type_encoded
0        Target                      1
1        Target                      1
2        Target                      1
3        Target                      1
4        Target                      1


In [11]:
df_merged_train["sequence_type_encoded"].value_counts()

sequence_type_encoded
1    344058
0    230887
Name: count, dtype: int64

In [12]:
df_merged_train = df_merged_train.sort_values(['sequence_id', 'sequence_counter'], ascending=[True, False])
df_merged_train.head()

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v62,tof_5_v63,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm,sequence_type_encoded
56,SEQ_000007_000056,Target,SEQ_000007,56,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Cheek - pinch skin,7.171875,...,96.0,94.0,0,12,1,1,163.0,52,24.0,1
55,SEQ_000007_000055,Target,SEQ_000007,55,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Cheek - pinch skin,6.101562,...,96.0,94.0,0,12,1,1,163.0,52,24.0,1
54,SEQ_000007_000054,Target,SEQ_000007,54,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Cheek - pinch skin,5.566406,...,98.0,95.0,0,12,1,1,163.0,52,24.0,1
53,SEQ_000007_000053,Target,SEQ_000007,53,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Cheek - pinch skin,6.144531,...,106.0,99.0,0,12,1,1,163.0,52,24.0,1
52,SEQ_000007_000052,Target,SEQ_000007,52,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Cheek - pinch skin,6.140625,...,101.0,93.0,0,12,1,1,163.0,52,24.0,1


In [13]:
X = df_merged_train[feature_list_with_seq_id_seq_counter]# assuming feature_list is your input columns
Y = df_merged_train[["sequence_id","sequence_type_encoded"]]
print(X.shape,Y.shape)

(574945, 341) (574945, 2)


In [14]:
Y.columns.to_list()

['sequence_id', 'sequence_type_encoded']

In [15]:
X.to_pickle("df_X.pkl")
Y.to_pickle("df_Y.pkl")


In [16]:
import numpy as np
import pandas as pd

def prepare_sequences(df,df_y ,max_steps=700):
    # Drop unnecessary columns early
  
    df = df.drop(columns=["sequence_counter"])  
    
    # Get number of features (exclude seq_id)
    feature_cols = [c for c in df.columns if c != "sequence_id"]
    print(feature_cols)
    n_features = len(feature_cols)
    
    # Find number of unique sequences
    sequence_ids = df["sequence_id"].unique()
    n_sequences = len(sequence_ids)

    # Preallocate final array (float32 to save memory)
    X = np.zeros((n_sequences, max_steps, n_features), dtype=np.float32)
    Y = np.zeros(n_sequences,)
    
    for i, seq_id in enumerate(sequence_ids):
        if(i%100==0):
            print(i)
        seq = df[df["sequence_id"] == seq_id][feature_cols].values.astype(np.float32)
        
        # Truncate if longer than max_steps, keep first 700
        if len(seq) > max_steps:
            seq = seq[:max_steps]
        
        # Pad if shorter
        X[i, :len(seq), :] = seq
        Y[i] = df_y.loc[df_y["sequence_id"] == seq_id, "sequence_type_encoded"].iloc[0]
    
    return X,Y




In [17]:
X,Y = prepare_sequences(X,Y)

['acc_x', 'acc_y', 'acc_z', 'rot_w', 'rot_x', 'rot_y', 'rot_z', 'thm_1', 'thm_2', 'thm_3', 'thm_4', 'thm_5', 'tof_1_v0', 'tof_1_v1', 'tof_1_v2', 'tof_1_v3', 'tof_1_v4', 'tof_1_v5', 'tof_1_v6', 'tof_1_v7', 'tof_1_v8', 'tof_1_v9', 'tof_1_v10', 'tof_1_v11', 'tof_1_v12', 'tof_1_v13', 'tof_1_v14', 'tof_1_v15', 'tof_1_v16', 'tof_1_v17', 'tof_1_v18', 'tof_1_v19', 'tof_1_v20', 'tof_1_v21', 'tof_1_v22', 'tof_1_v23', 'tof_1_v24', 'tof_1_v25', 'tof_1_v26', 'tof_1_v27', 'tof_1_v28', 'tof_1_v29', 'tof_1_v30', 'tof_1_v31', 'tof_1_v32', 'tof_1_v33', 'tof_1_v34', 'tof_1_v35', 'tof_1_v36', 'tof_1_v37', 'tof_1_v38', 'tof_1_v39', 'tof_1_v40', 'tof_1_v41', 'tof_1_v42', 'tof_1_v43', 'tof_1_v44', 'tof_1_v45', 'tof_1_v46', 'tof_1_v47', 'tof_1_v48', 'tof_1_v49', 'tof_1_v50', 'tof_1_v51', 'tof_1_v52', 'tof_1_v53', 'tof_1_v54', 'tof_1_v55', 'tof_1_v56', 'tof_1_v57', 'tof_1_v58', 'tof_1_v59', 'tof_1_v60', 'tof_1_v61', 'tof_1_v62', 'tof_1_v63', 'tof_2_v0', 'tof_2_v1', 'tof_2_v2', 'tof_2_v3', 'tof_2_v4', 'tof_2_v5

In [18]:
np.savez("provided_train_X_Y_np.npz", X=X, Y=Y)

In [19]:
data = np.load("provided_train_X_Y_np.npz") 
print(data["X"].shape) 
print(data["Y"])

(8151, 700, 339)
[1. 1. 1. ... 1. 1. 0.]


In [20]:
from sklearn.model_selection import train_test_split

# X = input data, y = labels
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y
)

In [21]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(6520, 700, 339) (6520,)
(1631, 700, 339) (1631,)


In [22]:
unique, counts = np.unique(Y_train, return_counts=True)
print(unique)
print(counts)

print(dict(zip(unique, counts)))

[0. 1.]
[2430 4090]
{np.float64(0.0): np.int64(2430), np.float64(1.0): np.int64(4090)}


In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking


num_features = 339
timesteps = 700

# Build LSTM model
model = Sequential([
    # Masking layer helps when sequences are shorter than 700 (if padded with 0s)
    Masking(mask_value=0.0, input_shape=(timesteps, num_features)),
    # LSTM layer (64 units). return_sequences=False means last output only.
    LSTM(64, return_sequences=False),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")   # for binary classification (adjust for regression/multiclass)
])

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",   # use "categorical_crossentropy" for multi-class
    metrics=[
        "accuracy", 
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)




2025-08-19 20:50:57.080864: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-19 20:50:57.095589: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-19 20:50:57.236539: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-19 20:50:57.372300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755616857.534652   24772 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755616857.56

In [24]:
# Train
history = model.fit(
    X_train, Y_train,
    batch_size=32,    # adjust to fit GPU memory
    epochs=20,
    validation_split=0.2,
    shuffle=True
)

# Save
history

Epoch 1/20


2025-08-19 20:51:09.266260: E tensorflow/core/util/util.cc:131] oneDNN supports DT_BOOL only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


163/163 ━━━━━━━━━━━━━━━━━━━━ 59s 352ms/step - accuracy: 0.6267 - loss: 0.6718 - precision: 0.6294 - recall: 0.9909 - val_accuracy: 0.6173 - val_loss: 0.6654 - val_precision: 0.6173 - val_recall: 1.0000
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 56s 345ms/step - accuracy: 0.6298 - loss: 0.6595 - precision: 0.6298 - recall: 1.0000 - val_accuracy: 0.6173 - val_loss: 0.6658 - val_precision: 0.6173 - val_recall: 1.0000
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 57s 349ms/step - accuracy: 0.6298 - loss: 0.6592 - precision: 0.6298 - recall: 1.0000 - val_accuracy: 0.6173 - val_loss: 0.6657 - val_precision: 0.6173 - val_recall: 1.0000
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 56s 346ms/step - accuracy: 0.6298 - loss: 0.6594 - precision: 0.6298 - recall: 1.0000 - val_accuracy: 0.6173 - val_loss: 0.6657 - val_precision: 0.6173 - val_recall: 1.0000
Epoch 5/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 57s 348ms/step - accuracy: 0.6298 - loss: 0.6593 - precision: 0.6298 - recall: 1.0000 - val_accuracy: 0.6173 - val_loss

KeyboardInterrupt: 

In [25]:
model_rnn = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(64, activation='tanh', input_shape=(timesteps, num_features)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_rnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",   # use "categorical_crossentropy" for multi-class
    metrics=[
        "accuracy", 
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)
history_rnn = model_rnn.fit(
    X_train, Y_train,
    batch_size=32,    # adjust to fit GPU memory
    epochs=20,
    validation_split=0.2,
    shuffle=True
)

# Save
history_rnn

/home/ankur/Desktop/ML_DL_Projects/ml_projects_venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


: 